<a href="https://colab.research.google.com/github/antoniosql/cursoml/blob/master/Ejercicio%20Churn/Prediciendo_del_abandono_de_clientes_Paso_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![SolidQ](https://www.solidq.com/wp-content/uploads/2015/06/Logo-SolidQ-Web.gif)
# Prediciendo el abandono de clientes


## Cómo leer ficheros de Drive en lugar de subirlos al entorno volátil de Colaboratory

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Tenemos un escenario en el que queremos medir el ratio de abandono (Churn Rate) de nuestros clientes de Telco. Es un concepto especialmente importante en negocios que están basados en suscripciones.

en nuestro conjunto de datos de ejemplo, cada ocurrencia representa un cliente de una compañía telefónica. Cada columna contiene atributos de ese cliente, como los cargos incurridos por servicio, tiempo de duración de su cuenta o si el cliente es todavía clinete o no. 

## Importamos las clases y módulos necesarios

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

## Explorando los datos

In [0]:

df_abandonos = pd.read_csv("/content/gdrive/My Drive/Datos/TelcoChurn.csv" )

In [0]:
df_abandonos.head()
#tenemos 3333 observaciones, de ellas 483 tienen Churn = yes

In [0]:
#filtro = df_abandonos["Churn"]=="yes"
#df_abandonos.where(filtro, inplace = False).count()


In [0]:
    correlacion = df_abandonos.corr() 
    fig, ax = plt.subplots(figsize=(21, 21)) 
    ax.matshow(correlacion) 
    plt.xticks(range(len(correlacion.columns)), correlacion.columns); 
    plt.yticks(range(len(correlacion.columns)), correlacion.columns); 

Es evidente que los cargos dependen de los minutos de conversación por lo que los pares de características x_calls x_Charge están muy correlacionadas entre sí

In [0]:

#Extraemos las etiquetas
resultado_abandono = df_abandonos['Churn']
y = np.where(resultado_abandono == 'yes',1,0)

# No necesitamos estas columnas
a_eliminar = ['State','Area_Code','Phone','Churn']
churn_feat_space = df_abandonos.drop(a_eliminar,axis=1)

yes_no_cols = ["Intl_Plan","Vmail_Plan"]
churn_feat_space[yes_no_cols] = churn_feat_space[yes_no_cols] == 'yes'


# Extrarmos características para su uso posterior

caracteristicas = churn_feat_space.columns

X = churn_feat_space.as_matrix().astype(np.float)

# Escalamos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

print ("Disponemos de %d observaciones y %d características" % X.shape)
print ("Etiquetas objetivo únicas:", np.unique(y))


## Validación cruzada de Algoritmos

In [0]:
from sklearn.model_selection import KFold

def run_cv(X,y,clf_class,**kwargs):
    # Construir el objeto Kfolds
    kf = KFold(5)
    y_pred = y.copy()

    # Iterar por los Folds
    for k, (train, test) in enumerate(kf.split(X, y)):
        X_train, X_test = X[train], X[test]
        y_train = y[train]
        # Inicializar un clasificador con los argumentos clave
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test] = clf.predict(X_test)
    return y_pred

### Probamos los tres algoritmos

In [0]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN

def accuracy(y_true,y_pred):
    # NumPy interprets True and False as 1. and 0.
    return np.mean(y_true == y_pred)

print ("Support vector machines:")
print ("%.3f" % accuracy(y, run_cv(X,y,SVC)))
print ("Random forest:")
print ("%.3f" % accuracy(y, run_cv(X,y,RF)))
print ("K-nearest-neighbors:")
print ("%.3f" % accuracy(y, run_cv(X,y,KNN)))

### Matriz de Confusión

In [0]:
from sklearn.metrics import confusion_matrix

y = np.array(y)


confusion_matrix(y,run_cv(X,y,SVC))


In [0]:
confusion_matrix(y,run_cv(X,y,RF)) 


In [0]:
confusion_matrix(y,run_cv(X,y,KNN))